In [7]:
from geopy.geocoders import Nominatim
import json
import pandas as pd
from geopy.geocoders import Nominatim
import time

In [2]:
locator = Nominatim(user_agent="Test Geocoder, contact:ivanangelov05@gmail.com")

In [3]:
location = locator.geocode("Rasho Dimitrov, Pernik, Bulgaria")

In [4]:
print(f"""
      Latitude: {location.latitude}
      Longitude: {location.longitude}
      Altitude: {location.altitude}
      """)


      Latitude: 42.6124279
      Longitude: 23.1003299
      Altitude: 0.0
      


In [18]:
# Load .JSON file
with open("dsk_offices_parsed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [19]:
# Collect results
results = []
for entry in data["entries"]:
    full_address = f'{entry["city"]}, {entry["address_line"]}'
    try:
        location = locator.geocode(full_address)
        if location:
            lat, lon = location.latitude, location.longitude
        else:
            lat, lon = None, None
    except Exception as e:
        lat, lon = None, None
    
    results.append({
        "otp_id": entry["otp_id"],
        "office_name": entry["office_name"],
        "city": entry["city"],
        "address": entry["address_line"],
        "lat": lat,
        "lon": lon
    })
    
    time.sleep(1)  # avoid rate limits

In [20]:
# Make a dataframe
df = pd.DataFrame(results)
display(df)

,otp_id,office_name,city,address,lat,lon
0,315,ВЛАДИСЛАВ,Варна,Варненчик 92,43.212944,27.905547
1,337,ПРЕСЛАВ,Варна,Преслав 69,43.203059,27.913274
2,2404,КАВАРНА,Каварна,Добротица 25,43.431666,28.337607
3,704,СЕВЛИЕВО,Севлиево,Свобода 9,43.056967,24.972563
4,401,Търново ВЕЛИКО ТЪРНОВО,Велико,Цар Освободител 3,43.132323,25.691136
5,701,ГАБРОВО,Габрово,Николаевска 17,42.864965,25.321132
6,1001,ЛОВЕЧ,Ловеч,Търговска 44,43.145258,24.716953
7,501,ВИДИН,Видин,Търговска 2,43.987175,22.876531
8,1101,МОНТАНА,Монтана,Трети март 62,43.819563,23.239884
9,601,ВРАЦА,Враца,Христо Ботев 44,43.144359,23.686956
